In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df_for_hire = pd.read_parquet("../input/fhv_tripdata_2021-01.parquet")

In [3]:
df_for_hire.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


## Q1: Number of records in Jan 2021 FHV data?

In [4]:
df_for_hire.shape

(1154112, 7)

### Q1 answer: 1154112

In [5]:
df_for_hire.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
dtype: object

In [6]:
df_for_hire['duration_in_sec'] = (df_for_hire['dropOff_datetime'] - df_for_hire['pickup_datetime']).dt.total_seconds()/60.0

## Q2: Average duration in Jan 2021 FHV?

In [7]:
df_for_hire['duration_in_sec'].mean()

19.1672240937939

### Q2 answer: 19.16

## Q3: Fraction of missing values in column PUlocationID

In [8]:
df_for_hire['PUlocationID'].isna().value_counts()

True     958267
False    195845
Name: PUlocationID, dtype: int64

In [9]:
df_for_hire['DOlocationID'].isna().value_counts()

False    991892
True     162220
Name: DOlocationID, dtype: int64

In [10]:
df_for_hire['PUlocationID'].fillna(-1.0,inplace=True)
df_for_hire['DOlocationID'].fillna(-1.0,inplace=True)

In [11]:
((df_for_hire['PUlocationID']== -1.0).sum()/df_for_hire.shape[0])*100

83.03067639882438

### Q3 answer: 83.03%

## Q4: Dimensionality after OHE. Number of columns

In [12]:
df_for_hire = df_for_hire.loc[(df_for_hire['duration_in_sec'] >= 1) & (df_for_hire['duration_in_sec'] <= 60)]

In [13]:
%%time
features = ['PUlocationID', 'DOlocationID']
df_for_hire[features] = df_for_hire[features].astype(str)
dv = DictVectorizer()
features_dict = df_for_hire[features].to_dict(orient='records')

CPU times: user 3.58 s, sys: 373 ms, total: 3.95 s
Wall time: 3.93 s


In [14]:
%%time
X_train = dv.fit_transform(features_dict)

CPU times: user 2.42 s, sys: 111 ms, total: 2.53 s
Wall time: 2.52 s


In [15]:
X_train.shape

(1109826, 525)

### Q4 answer: 525

## Q5: Calculate RMSE on X_train

In [16]:
%%time
target = 'duration_in_sec'
y_train = df_for_hire[target].to_numpy()
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

CPU times: user 37 s, sys: 1.97 s, total: 39 s
Wall time: 9.8 s


10.528519107212292

### Q5 answer: 10.52

## Q6: Calculate RMSE on X_valid

In [17]:
%%time
df_feb2021 = pd.read_parquet('../input/fhv_tripdata_2021-02.parquet')
df_feb2021['duration_in_sec'] = (df_feb2021['dropOff_datetime'] - df_feb2021['pickup_datetime']).dt.total_seconds()/60.0

df_feb2021 = df_feb2021.loc[(df_feb2021['duration_in_sec'] >= 1) & (df_feb2021['duration_in_sec'] <= 60)]


CPU times: user 353 ms, sys: 149 ms, total: 502 ms
Wall time: 290 ms


In [18]:
%%time
df_feb2021['PUlocationID'].fillna(-1.0,inplace=True)
df_feb2021['DOlocationID'].fillna(-1.0,inplace=True)
df_feb2021[features] = df_feb2021[features].astype(str)
test_dict = df_feb2021[features].to_dict(orient='records')

X_valid = dv.transform(test_dict)
y_valid = df_feb2021[target].to_numpy()

CPU times: user 5.39 s, sys: 294 ms, total: 5.69 s
Wall time: 5.69 s


In [19]:
y_pred1 = lr.predict(X_valid)

mean_squared_error(y_valid, y_pred1, squared=False)

11.014283211122269

### Q6 answer: 11.01

In [20]:
import pickle

In [21]:
with open('models/lin_reg.pickle', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)